### Document embedding

In [1]:
import pandas as pd

#### load supermarket dataset

In [2]:
df = pd.read_csv('./productes/short_raw_supermarket.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457 entries, 0 to 456
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   category_id  457 non-null    object 
 1   internal_id  457 non-null    int64  
 2   description  457 non-null    object 
 3   format       452 non-null    object 
 4   weight       299 non-null    object 
 5   brand        457 non-null    object 
 6   units        457 non-null    int64  
 7   image_loc    457 non-null    object 
 8   price_date   457 non-null    object 
 9   price        457 non-null    float64
 10  pricebase    330 non-null    float64
 11  promotion    395 non-null    object 
 12  source_id    457 non-null    int64  
 13  source_name  457 non-null    object 
dtypes: float64(2), int64(3), object(9)
memory usage: 50.1+ KB


In [3]:
col_names = df.columns[[2, 3, 4, 5, 6, 8, 9, 13]]

In [4]:
df[col_names]

,description,format,weight,brand,units,price_date,price,source_name
0,FONT VELLA AGUA MINERAL NATURAL BOTELLA 33 CL,33 CL,8 L,FONT VELLA,1,2023-03-12,0.95,DIA
1,LANJARON AGUA MINERAL NATURAL BOTELLA TAPÓN SP...,33 CL,330ML,LANJARON,1,2023-03-12,0.65,DIA
2,"AGUA MINERAL NATURAL KIDS ZOO LANJARON, BOTELL...",33 CL,NaN,LANJARON,1,2023-03-12,0.75,CAPRABO
3,FONTAREL ZERO SODIO AGUA MINERAL NATURAL SIN G...,"1,50 L",NaN,FONTAREL,1,2023-03-20,0.65,ECI
4,PERRIER AGUA MINERAL CON GAS BOTELLA 50 CL,50 CL,1 L,PERRIER,1,2023-03-12,1.13,DIA
...,...,...,...,...,...,...,...,...
452,"ACEITE LABIAL GINSENG 2 ESSENCE, PACK 1 UD",G 2,NaN,ESSENCE,1,2023-03-08,2.99,EROSKI
453,BARRA DE LABIOS COLOR SENSATIONAL 211 MAYBELLI...,"4,2 G",NaN,MAYBELLINE,1,2023-03-08,5.99,EROSKI
454,"AGUA DE COLONIA AGUA FRESCA ALVAREZ GÓMEZ, SPR...",BOTELLA,750 ML,ALVAREZ GÓMEZ,1,2023-03-12,6.89,CAPRABO
455,"HERVIDOR DE AGUA TAURUS VINTAGE RED, CAPACIDAD...",50 CL,5 L,TAURUS VINTAGE RED,1,2023-03-12,29.90,ALCAMPO


In [5]:
df['description'] = df.description.astype('str')

- tenim el mateix producte amb diferents noms a diferents productes i els volem relacionar. 
- no anem a fer IA per fer això.
- les descripcions, les considerem diferents textos i fem un embeding de diferents textos. 
- productes que són el mateix però es diuen diferents, però suposem que són iguals encara que tingui descrpcions diferents (és l'heuristica). 
- fem un embedding, de les descripsions i fem un most similars i torna els deu més similars. 
- com a heurística, ens serveix per solucionar el problema?

#### preprocess data

In [6]:
from gensim.utils import simple_preprocess

In [7]:
preprocessed_data = df.description.apply(simple_preprocess)
preprocessed_data

0      [font, vella, agua, mineral, natural, botella,...
1      [lanjaron, agua, mineral, natural, botella, ta...
2      [agua, mineral, natural, kids, zoo, lanjaron, ...
3      [fontarel, zero, sodio, agua, mineral, natural...
4        [perrier, agua, mineral, con, gas, botella, cl]
                             ...                        
452         [aceite, labial, ginseng, essence, pack, ud]
453    [barra, de, labios, color, sensational, maybel...
454    [agua, de, colonia, agua, fresca, alvarez, góm...
455    [hervidor, de, agua, taurus, vintage, red, cap...
456               [aceite, en, crema, oliva, mítica, ml]
Name: description, Length: 457, dtype: object

- si fem un embeding d'això: 
    - agua mineral: most similar. 
    - tokenització més fina: en comptes de aguas -> agua
- el que pasa es que el input, per trobar la similitud, primer l'hem de pasar al embeding. i per fer l'embeding l'hem de tokenitzar -> fer inferència i després podem veure. si l'usuari posa aguas, el tokenitzador convertirà a agua. 

#### tag documents

In [8]:
from gensim.models.doc2vec import TaggedDocument

In [9]:
tagged_data = [TaggedDocument(words = words, tags = [str(i)]) for i, words in enumerate(preprocessed_data)]
tagged_data[0]

TaggedDocument(words=['font', 'vella', 'agua', 'mineral', 'natural', 'botella', 'cl'], tags=['0'])

- tagged, que identifiqui cada document. 
- work_to_vec, el tag es la propia paraula.
- aqui necessitem afegir un tag. Enumerem i li posem un id. 

#### Doc2Vec model

In [10]:
from gensim.models.doc2vec import Doc2Vec

In [11]:
d2v = Doc2Vec(dm = 0, vector_size = 10, window = 20, alpha = 0.025, workers =4)
d2v

- dm: training algorithm (0: PV-DBOW, 1: PV-DM)
- vector_size: dimension of the feature vectors
- window: max. distance between related words in a sentence
- alpha: learning rate
- min_count: ignore words with frequency lower than this
- max_vocab_size: unique words (10M words~ 1GB, prune infrequent words if not enough memory)
- --
- els mateixos parametres que abans. 
- window, vol dri que totes les paraules es relacionen amb totes perquè són frases curtes. 
- vector_size: dimensions del embeding
- dm: especifica el tipus d'algoritme que farem servir per fer el embeding (vàries heuristiques). 

In [12]:
d2v.vector_size, d2v.window, d2v.alpha, d2v.workers, d2v.dm

(10, 20, 0.025, 4, False)

#### build document corpus

In [13]:
d2v.build_vocab(tagged_data)

In [14]:
d2v.corpus_count, d2v.corpus_total_words

(457, 3432)

#### train model

In [15]:
%%time
d2v.train(tagged_data, total_examples =d2v.corpus_count, epochs = 50)

CPU times: user 74.7 ms, sys: 64.7 ms, total: 139 ms
Wall time: 231 ms


#### word vectors and document vectors

In [16]:
d2v.wv, d2v.dv

(<gensim.models.keyedvectors.KeyedVectors at 0x7f2afae78e20>,
 <gensim.models.keyedvectors.KeyedVectors at 0x7f2afae79540>)

- cada paraula té el seu vector i cada document té el seu vector també. 

In [17]:
len(d2v.wv), len(d2v.dv)

(145, 457)

In [18]:
d2v.wv['botella'], d2v.dv['0']

(array([-0.0960355 ,  0.05007293, -0.08759586, -0.04391825, -0.000351  ,
        -0.00296181, -0.0766124 ,  0.09614743,  0.04982058,  0.09233143],
       dtype=float32),
 array([ 0.37936684,  0.32132193, -0.19728473,  0.14251502,  0.0678161 ,
         0.06681497, -0.21811473, -0.6779954 , -0.4435215 ,  0.23385449],
       dtype=float32))

#### inference

In [19]:
d2v.infer_vector(preprocessed_data[0])

array([ 0.33170772,  0.3640848 , -0.19806907,  0.0145197 ,  0.03978547,
        0.0619348 , -0.13480988, -0.7682383 , -0.3888529 ,  0.24043772],
      dtype=float32)

- Una paraula que no hagi vist al entrenament, serà com una sèrie de caracters random xd. 
- Per això el ground truth és tant important (corpus).

#### retrieve similar sentences

In [20]:
query = d2v.infer_vector(['agua', 'mineral', 'carrefour', 'con', 'limón', 'pomelo'])
mss = d2v.dv.most_similar(query, topn =5)
preprocessed_data[10], mss

(['agua', 'mineral', 'carrefour', 'con', 'limón', 'pomelo'],
 [('255', 0.9948669672012329),
  ('3', 0.992652416229248),
  ('272', 0.9918926954269409),
  ('364', 0.991263210773468),
  ('179', 0.990940511226654)])

In [21]:
df.iloc[[int(index) for index, similarity in mss], ]

,category_id,internal_id,description,format,weight,brand,units,image_loc,price_date,price,pricebase,promotion,source_id,source_name
255,D40-41-02,13036,NESCAFÉ CLASSIC NATURAL - CAFÉ SOLUBLE 100G,100 G,200 G,NESCAFÉ,1,https://scrape.codeworks.es/imgs/7/156906_3.jpg,2023-03-12,4.30,7.89,70% DESCUENTO EN LA 2ª UNIDAD,7,CAPRABO
3,B00-01-30,60913,FONTAREL ZERO SODIO AGUA MINERAL NATURAL SIN G...,"1,50 L",NaN,FONTAREL,1,https://scrape.codeworks.es/imgs/10/0011863000...,2023-03-20,0.65,NaN,5€ DE DESCUENTO POR COMPRA SUPERIOR A 20€. INT...,10,ECI
272,D40-51-00,15237,"BARRITA DE YOGUR-LIMÓN SUSTITUYE, CAJA 128 G",128 G,NaN,SUSTITUYE,1,https://scrape.codeworks.es/imgs/8/13289236.jpg,2023-03-10,4.79,NaN,2ª UNIDAD -70 %,8,EROSKI
364,D70-60-07,11665,"PREPARADO LACTEO DESNATADO, ENRIQUECIDO CON AV...",1 L,NaN,PULEVA OMEGA 3,1,https://scrape.codeworks.es/imgs/6/12045548257...,2023-03-15,1.78,1.95,"ABANS 1,95€",6,ALCAMPO
179,D21-01-07,62652,"CREMA DE MARISCO DESHIDRATADO KNORR GOURMET, S...",72 G,NaN,KNORR,1,https://scrape.codeworks.es/imgs/8/25522467.jpg,2023-03-27,1.80,12.95,2ª UNIDAD -50 %,8,EROSKI


#### check

In [22]:
i = 10
preprocessed_data[i], d2v.dv[i]

(['agua', 'mineral', 'carrefour', 'con', 'limón', 'pomelo'],
 array([ 0.35393962,  0.32939717, -0.11443933,  0.09357429, -0.02630674,
         0.02121589, -0.13578935, -0.46138144, -0.23472516,  0.221645  ],
       dtype=float32))

In [23]:
query = d2v.dv[i]
d2v.dv.most_similar(query, topn = 1)

[('10', 1.0)]

In [24]:
query = d2v.infer_vector(['agua', 'mineral', 'carrefour', 'con', 'limón', 'pomelo'])
query

array([ 0.39885235,  0.42710617, -0.01862206, -0.0024645 ,  0.00860504,
        0.05370666, -0.12931995, -0.54688865, -0.31032833,  0.1855152 ],
      dtype=float32)

In [25]:
d2v.dv.most_similar(query, topn = 1)

[('255', 0.99543696641922)]

- si vas a buscar on has tret aquest, punt, si fas una query amb aquest vector, ja el troba, perquè es exactament aquell vector. 
- si el que li fem és fes-me inferència de la descripció de nou, genera un embeding/vector, que no es el que tens al diccionari.
- després ho fa malament. 
- quan li passem un input, que en el moment d'entrenament ha tornat un vector, si fem inferència amb el mateix input, torna un altre vector. 
- els pesos no estàn ben ajustats.
- algun parametre esta malament. 
- ell l'ha situat en algun punt però està malament. 
- ---
- exemple pràctic dels embedings.
- posiblement, ja tingui els batch fets. (exprimir els recursos). 